In [34]:
import gspread
import pandas as pd

import numpy as np
import time
import re
import json
import csv
import math

from gspread_dataframe import set_with_dataframe
from selenium import webdriver
from pandas import DataFrame as df

import setting


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from google.oauth2.service_account import Credentials
from webdriver_manager.chrome import ChromeDriverManager


In [29]:
with open('setting.json') as f:
    df = json.load(f)
    SPREADSHEET_KEY = df['KANAGAWA_SPREADSHEET_KEY']
    SECRET_KEY_PATH = df['SECRET_KEY_PATH']

scope = ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(
    SECRET_KEY_PATH, scopes=scope)
gc = gspread.authorize(credentials)
workbook = gc.open_by_key(SPREADSHEET_KEY)

In [30]:
driver = webdriver.Chrome(ChromeDriverManager().install())




====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/tatemukaiyuki/.wdm/drivers/chromedriver/mac64/102.0.5005.61]


In [ ]:
def get_list(i):
    URL =  'https://beauty.hotpepper.jp/esthe/svcSA/macJP/salon/PN'+ str(i) +'.html?searchGender=ALL'
    driver.get(url)

In [ ]:
def main():
    url = 'https://beauty.hotpepper.jp/esthe/svcSA/macAU/salon/'
    driver.get(url)

    total_items = driver.find_element_by_css_selector('.numberOfResult').text
    
    page_nums = math.ceil(int(total_items) / 20)
    
    print(total_items + '件取得します')
    print(page_nums + 'ページ')
    
    df_list = []
    for i in range(1, page_nums+1):
        
        href_list, title_list = get_list(i)
        
        df = pd.DataFrame({
            '店名':title_list,
            'URL':href_list
        })
        df_list.append(df)
        
    df_concat = pd.concat(df_list).reset_index(drop = True)
    
    with open('dataframe.pickle', 'wb') as f:
        pickle.dump(df_concat, f)
    
    #スプレッドシートに書く
    sheet_name = '両国・錦糸町・小岩・森下・瑞江'
    gc = Spreadsheet(sheet_name, title='HOTPEPPER')
    gc.update_sheet(df_concat)
    
    wd.driver.quit()
    print('終了')

In [35]:
#debug

url = 'https://beauty.hotpepper.jp/esthe/svcSA/macAU/salon/'
driver.get(url)

total_items = driver.find_element_by_css_selector('.numberOfResult').text
page_nums = math.ceil(int(total_items) / 20)

In [39]:
title_list = ["あ", "い","う", "え","お"]
href_list = ["a", "i","u", "e","o"]

df = pd.DataFrame({
    '店名':title_list,
    'URL':href_list
})

In [40]:
df

,店名,URL
0,あ,a
1,い,i
2,う,u
3,え,e
4,お,o
